# Running inference tools

As machine learning (ML) becomes more popular in HEP analysis, coffea also
provide tools to assist with using ML tools within the coffea framework. For
training and validation, you would likely need custom data mangling tools to
convert standard CMS data formats ([NanoAOD][nanoaod], [PFNano][pfnano]) to a
format that best interfaces with the ML tool of choice to have fine control over
what is done. For more advanced use cases of data mangling and data saving,
refer to the [awkward array manual][datamangle] and
[uproot][uproot_write]/[parquet][ak_parquet] write operations for saving
intermediate states. This reference mainly focuses on the result inference side
of ML tools, where ML tool outputs are used as another variable to be used in
the event/object selection chain.

[nanoaod]: https://twiki.cern.ch/twiki/bin/view/CMSPublic/WorkBookNanoAOD
[pfnano]: https://github.com/cms-jet/PFNano
[datamangle]: https://awkward-array.org/doc/main/user-guide/how-to-restructure.html
[uproot_write]: https://uproot.readthedocs.io/en/latest/basic.html#writing-ttrees-to-a-file
[ak_parquet]: https://awkward-array.org/doc/main/reference/generated/ak.to_parquet.html


## Why these wrapper tools are needed

The typical operation of using ML inference tools in the awkward/coffea analysis
tools involves the conversion and padding of awkward array to ML tool containers
(usually something that is `numpy`-compatible), run the inference, then
convert-and-truncate back into the awkward array syntax required for the
analysis chain to continue. With awkward arrays' laziness now being handled
entirely by [`dask`][dask_awkward], the conversion operation of awkward array to
other array types needs to be wrapped in a way that is understandable to `dask`.
The packages in the `ml_tools` package attempts to wrap the common tools used by
the HEP community with a common interface to reduce the verbosity of the code on
the analysis side.

[dask_awkward]: https://dask-awkward.readthedocs.io/en/stable/gs-limitations.html



## Example using ParticleNet-like jet variable calculation using PyTorch

The example given in this notebook be using [`pytorch`][pytorch] to calculate a
jet-level discriminant using its constituent particles. An example for how to
construct such a `pytorch` network can be found in the docs file, but for
`mltools` in coffea, we only support the [TorchScript][pytorch] format files to
load models to ensure operability when scaling to clusters. Let us first start
by downloading the example ParticleNet model file and a small PFNano compatible
file.


[pytorch]: https://pytorch.org/
[pytorch_jit]: https://pytorch.org/tutorials/beginner/saving_loading_models.html#export-load-model-in-torchscript-format


In [1]:
!wget --quiet -O model.pt https://github.com/CoffeaTeam/coffea/raw/ml_tools/tests/samples/triton_models_test/pn_test/1/model.pt

!wget --quiet -O pfnano.root https://github.com/CoffeaTeam/coffea/raw/ml_tools/tests/samples/pfnano.root


Here we write a simple function for opening events, as well as create a simple
function for creating a 4-vector compatible `Jet.pfCands` collection from the
default `Jet.constituents`

In [2]:
from coffea.nanoevents import NanoEventsFactory
from coffea.nanoevents.schemas import PFNanoAODSchema


def open_events(permit_dask=False):
    import awkward
    import dask_awkward

    ak = dask_awkward if permit_dask else awkward

    events = NanoEventsFactory.from_root(
        "file:./pfnano.root",
        schemaclass=PFNanoAODSchema,
        permit_dask=permit_dask,
    ).events()
    jet = events.Jet
    PFCands = events.PFCands[events.JetPFCands.pFCandsIdx]
    njets = ak.count(jet.pt, axis=-1)
    ncands = ak.flatten(jet.nConstituents)
    jet["PFCands"] = ak.unflatten(ak.unflatten(ak.flatten(PFCands), ncands), njets)
    events["Jet"] = jet
    return events


Now we prepare a class to handle inference request. Here we import the base
`torch_wrapper` class and create a new class that inherits `torch_wrapper`. As
the class cannot know anything about the data mangling required, we will need to
overload at least the method `awkward_to_numpy`:

- The input can be an arbitrary number of awkward arrays. Here we will be
  passing in the event array in the format constructed as the format array.
- The output should be single tuple `a` and single dictionary `b`, this is to
  ensure that arbitrarily complicated outputs can be passed to the underlying
  `pytorch` model instance like `model(*a, **b)`. The contents of `a` and `b`
  will need to be determined by the model of interest. In this ParticleNet-like
  example, the model expects the following inputs:

  - A `N` jets x `2` coordinate x `100` constituents "points" array,
    representing the constituent coordinates.
  - A `N` jets x `5` feature x `100` constituents "features" array, representing
    the constituent features of interest to be used for inference.
  - A `N` jets x `1` mask x `100` constituent "mask" array, representing whether
    a constituent should be masked from the inference request.

  The user is also responsible to making sure the data format is compatible with
  the model of interest. Defining this minimum class, we can attempt to run an
  inference using the `__call__` method of our defined class.



In [3]:
from coffea.ml_tools.torch_wrapper import torch_wrapper
import awkward as ak
import numpy as np


class ParticleNetExample1(torch_wrapper):
    def awkward_to_numpy(self, events):
        jets = ak.flatten(events.Jet)

        def pad(arr):
            return ak.fill_none(
                ak.pad_none(arr, 100, axis=1, clip=True),
                0.0,
            )

        # Human readable version of what the inputs are
        # Each array is a N jets x 100 constituent array
        imap = {
            "points": {
                "deta": pad(jets.eta - jets.PFCands.eta),
                "dphi": pad(jets.delta_phi(jets.PFCands)),
            },
            "features": {
                "dr": pad(jets.delta_r(jets.PFCands)),
                "lpt": pad(np.log(jets.PFCands.pt)),
                "lptf": pad(np.log(jets.PFCands.pt / jets.pt)),
                "f1": pad(np.log(np.abs(jets.PFCands.d0) + 1)),
                "f2": pad(np.log(np.abs(jets.PFCands.dz) + 1)),
            },
            "mask": {
                "mask": pad(ak.ones_like(jets.PFCands.pt)),
            },
        }

        # Compacting the array elements into the desired dimension using
        # ak.concatenate
        retmap = {
            k: ak.concatenate([x[:, np.newaxis, :] for x in imap[k].values()], axis=1)
            for k in imap.keys()
        }

        # Returning everything using a dictionary. Also take care of type
        # conversion here.
        return (), {
            "points": ak.values_astype(retmap["points"], "float32").to_numpy(),
            "features": ak.values_astype(retmap["features"], "float32").to_numpy(),
            "mask": ak.values_astype(retmap["mask"], "float16").to_numpy(),
        }


pn_example1 = ParticleNetExample1("model.pt")
events = open_events(permit_dask=False)
results = pn_example1(events)
print(results)
print(type(results))
print(results.__repr__)


/home/ensc/VirtualENV/coffea-test/lib/python3.8/site-packages/coffea/ml_tools/helper.py:155: UserWarning: No format checks were performed on input!
  warnings.warn("No format checks were performed on input!")


[[0.0693, -0.0448], [0.0678, -0.0451], ..., [0.0616, ...], [0.0587, -0.0172]]
<class 'awkward.highlevel.Array'>
<bound method Array.__repr__ of <Array [[0.0693, -0.0448], ..., [0.0587, -0.0172]] type='64 * 2 * float32'>>


For each jet in the input to the `torch` model, the model returns a 2-tuple
probability value. Without additional specification, the `torch_wrapper` class
performs a trival conversion of `ak.from_numpy` of the torch model's output. We
can specify that we want to fold this back into nested structure by overloading
the `numpy_to_awkward` method of the class. 

For this example we are going perform additional computation for the conversion
back to awkward array formats: 

- Calculate the `softmax` method for the return of each jet (commonly used for
  ML output ``scores'')
- Fold the computed `softmax` array back into nested structure that is
  compatible with the original inputs events array.

Notice that the inputs of the `numpy_to_awkward` method is different from the
`awkward_to_numpy` method, only by that the first argument is the return `numpy`
array of the model inference.


In [4]:
class ParticleNetExample2(ParticleNetExample1):
    def numpy_to_awkward(self, return_array, events):
        softmax = np.exp(return_array)[:, 0] / np.sum(np.exp(return_array), axis=-1)

        njets = ak.count(events.Jet.pt, axis=-1)
        return ak.unflatten(softmax, njets)


pn_example2 = ParticleNetExample2("model.pt")
jets = events.Jet
jets["MLresults"] = pn_example2(events)
events["Jet"] = jets

print(events.Jet.pt)
print(events.Jet.MLresults)


[[1.68e+03, 1.41e+03, 298, 85.9, 32.7, 16.3, 16.2, 15.8], ..., [1.58e+03, ...]]
[[0.528, 0.528, 0.524, 0.523, 0.521, 0.52, 0.519, 0.519], ..., [0.528, ...]]


Now we have a per-jet variable we can use to continue the event/object selection
chain for our analysis! 

Notice that up till now, we have been working exclusively with plain awkward.
But this class is already ready to be extended to dask awkward! The `__call__`
method commonly knows how to handle the different array types.


In [5]:
import dask_awkward as dak

# Creating a lazy dask array of events array with the same
dask_events = open_events(permit_dask=True)

# Syntax for dask arrays is identical to the plain awkward arrays!
dask_jets = dask_events.Jet
dask_jets["MLresults_dask"] = pn_example2(dask_events)
dask_events["Jet"] = dask_jets

# Checking that we get identical results
print(dask_events.Jet.MLresults_dask.compute())
print(ak.all(dask_events.Jet.MLresults_dask.compute() == events.Jet.MLresults))

# Check which columns are loaded
print(dak.necessary_columns(dask_events.Jet.MLresults_dask))


/home/ensc/VirtualENV/coffea-test/lib/python3.8/site-packages/dask_awkward/lib/structure.py:751: UserWarning: Please ensure that dask.awkward<flatten, npartitions=1>
        is partitionwise-compatible with dask.awkward<flatten, npartitions=1>
        (e.g. counts comes from a dak.num(array, axis=1)),
        otherwise this unflatten operation will fail when computed!
  warnings.warn(
/home/ensc/VirtualENV/coffea-test/lib/python3.8/site-packages/dask_awkward/lib/structure.py:751: UserWarning: Please ensure that dask.awkward<count, npartitions=1>
        is partitionwise-compatible with dask.awkward<unflatten, npartitions=1>
        (e.g. counts comes from a dak.num(array, axis=1)),
        otherwise this unflatten operation will fail when computed!
  warnings.warn(
/home/ensc/VirtualENV/coffea-test/lib/python3.8/site-packages/coffea/ml_tools/helper.py:306: UserWarning: Touching all columns recursively may have performance penaltiesConsider overloading the dask_columns method of Particl

[[0.528, 0.528, 0.524, 0.523, 0.521, 0.52, 0.519, 0.519], ..., [0.528, ...]]
True
{'from-uproot-7a69c3c00169cd8f8875c60e974fcac3': ['L1.IsoTau40er_ETM105', 'HLT.PFJetFwd60', 'FsrPhoton.muonIdx', 'IsoTrack.pfRelIso03_chg', 'FatJet.n2b1', 'Jet.nCHadrons', 'HLT.L2Mu40_NoVertex_3Sta_NoBPTX3BX', 'FatJet.DDX_tau2_flightDistance2dSig', 'SubGenJetAK8.pt', 'GenPart.distinctChildrenDeepIdxG', 'FatJetPFCands.pFCandsIdx', 'HLT.Ele24_eta2p1_WPTight_Gsf_LooseChargedIsoPFTau30_eta2p1_CrossL1', 'MET.sumEt', 'L1.ETT50_BptxAND', 'HLT.Dimuon25_Jpsi', 'GenFatJetSVs.jetIdx', 'L1.DoubleJet60er2p7_ETM90', 'L1.SingleMu25', 'L1.SingleMuCosmics_EMTF', 'HLT.Dimuon0_Jpsi_NoVertexing_L1_4R_0er1p5R', 'HLT.Ele200_CaloIdVT_GsfTrkIdT', 'L1.DoubleJet30_Mass_Min340_dEta_Max1p5', 'L1.SingleIsoEG34', 'Electron.mvaFall17V1noIso_WPL', 'L1.SingleMu11_LowQ', 'SubJet.phi', 'HLT.Mu8_TrkIsoVVL_Ele23_CaloIdL_TrackIdL_IsoVL_DZ', 'HLT.AK8PFJetFwd400', 'PFCands.pvAssocQuality', 'HLT.Mu50', 'HLT.AK8PFJet140', 'Tau.decayMode', 'HLT.Do

The only remaining issue is that `dask` is currently loading all columns in the
array that is passed to the `awkward_to_numpy` method, even those that are not
required for the ML inference computation. While this behavior ensure that
everything is loaded, leading to less room for errors to during the development
of the `awkward_to_numpy` method, leaving this behavior unchanged can load to
excessive memory usage, depending on how the event array is set up.

To properly solve this behavior, one should further overload the `dask_columns`
method to return a list of columns that is strictly needed by the inference
tools. The inputs should be identical to that of the `awkward_to_numpy` method
defined by the user. Notice how this cell is considerably faster than the
previous. 

*Note:* as of writing, there is a [bug][bug] that causes the kernel to
segmentation fault if the required branches are not included. If you are running
into issues, it might be that you need to allow the class you are using to be
slightly less lazy.

[bug]: https://github.com/dask-contrib/dask-awkward/issues/249


In [6]:
class ParticleNetExample3(ParticleNetExample2):
    def dask_columns(self, events):
        return [
            events.Jet.pt,
            events.Jet.eta,
            events.Jet.phi,
            events.Jet.PFCands.pt,
            events.Jet.PFCands.eta,
            events.Jet.PFCands.phi,
            events.Jet.PFCands.d0,
            events.Jet.PFCands.dz,
        ]


pn_example3 = ParticleNetExample3("model.pt")

# Reloading a lazy instance of the array
dask_events = open_events(permit_dask=True)

# Syntax for dask arrays is identical to the plain awkward arrays!
dask_jets = dask_events.Jet
dask_jets["MLresults_lazy"] = pn_example3(dask_events)
dask_events["Jet"] = dask_jets

# Checking that we get identical results
print(dask_events.Jet.MLresults_lazy.compute())
# Checking that we get identical results
print(ak.all(dask_events.Jet.MLresults_lazy.compute() == events.Jet.MLresults))

# Check which columns are loaded
print(dak.necessary_columns(dask_events.Jet.MLresults_lazy))


[[0.528, 0.528, 0.524, 0.523, 0.521, 0.52, 0.519, 0.519], ..., [0.528, ...]]
True
{'from-uproot-f8512f447b3981f81867e04375f667d4': ['Jet.nConstituents', 'PFCands.d0', 'PFCands.dz', 'PFCands.pt', 'Jet.eta', 'Jet.pt', 'Jet.phi', 'JetPFCands.pFCandsIdx', 'PFCands.eta', 'PFCands.phi']}


Of course, the implementation of the classes above can be written in a single
class. Here is a copy-and-paste implementation of the class with all the
functionality described in the cells above:

In [7]:
class ParticleNetExample(torch_wrapper):
    def awkward_to_numpy(self, events):
        jets = ak.flatten(events.Jet)

        def pad(arr):
            return ak.fill_none(
                ak.pad_none(arr, 100, axis=1, clip=True),
                0.0,
            )

        # Human readable version of what the inputs are
        # Each array is a N jets x 100 constituent array
        imap = {
            "points": {
                "deta": pad(jets.eta - jets.PFCands.eta),
                "dphi": pad(jets.delta_phi(jets.PFCands)),
            },
            "features": {
                "dr": pad(jets.delta_r(jets.PFCands)),
                "lpt": pad(np.log(jets.PFCands.pt)),
                "lptf": pad(np.log(jets.PFCands.pt / jets.pt)),
                "f1": pad(np.log(np.abs(jets.PFCands.d0) + 1)),
                "f2": pad(np.log(np.abs(jets.PFCands.dz) + 1)),
            },
            "mask": {
                "mask": pad(ak.ones_like(jets.PFCands.pt)),
            },
        }

        # Compacting the array elements into the desired dimension using
        # ak.concatenate
        retmap = {
            k: ak.concatenate([x[:, np.newaxis, :] for x in imap[k].values()], axis=1)
            for k in imap.keys()
        }

        # Returning everything using a dictionary. Also take care of type
        # conversion here.
        return (), {
            "points": ak.values_astype(retmap["points"], "float32").to_numpy(),
            "features": ak.values_astype(retmap["features"], "float32").to_numpy(),
            "mask": ak.values_astype(retmap["mask"], "float16").to_numpy(),
        }

    def numpy_to_awkward(self, return_array, events):
        softmax = np.exp(return_array)[:, 0] / np.sum(np.exp(return_array), axis=-1)

        njets = ak.count(events.Jet.pt, axis=-1)
        return ak.unflatten(softmax, njets)

    def dask_columns(self, events):
        return [
            events.Jet.pt,
            events.Jet.eta,
            events.Jet.phi,
            events.Jet.PFCands.pt,
            events.Jet.PFCands.eta,
            events.Jet.PFCands.phi,
            events.Jet.PFCands.d0,
            events.Jet.PFCands.dz,
        ]


pn_example = ParticleNetExample("model.pt")

# Reloading a lazy instance of the array
dask_events = open_events(permit_dask=True)

# Syntax for dask arrays is identical to the plain awkward arrays!
dask_jets = dask_events.Jet
dask_jets["MLresults"] = pn_example(dask_events)
dask_events["Jet"] = dask_jets

# Checking that we get identical results
print(dask_events.Jet.MLresults.compute())

# Check which columns are loaded
print(dak.necessary_columns(dask_events.Jet.MLresults))


[[0.528, 0.528, 0.524, 0.523, 0.521, 0.52, 0.519, 0.519], ..., [0.528, ...]]
{'from-uproot-00618dded7651437e4e71167a590ec14': ['Jet.nConstituents', 'PFCands.d0', 'PFCands.dz', 'PFCands.pt', 'Jet.eta', 'Jet.pt', 'Jet.phi', 'JetPFCands.pFCandsIdx', 'PFCands.eta', 'PFCands.phi']}


## Comments about generalizing to other ML tools

All ML wrappers provided in the `coffea.mltools` module (`triton_wrapper` for
[triton][triton] server inference, `torch_wrapper` for pytorch, and
`xgboost_wrapper` for [xgboost][xgboost] inference) follow the same design:
analyzers is responsible for providing the model of interest, along with
providing an inherited class that overloads of the following methods to data
type conversion:

- `awkward_to_numpy`: converting awkward arrays to `numpy` arrays, the output
  `numpy` arrays should be in the format of a tuple `a` and a dictionary `b`,
  which can be expanded out to the input of the ML tool like `model(*a, **b)`.
  Notice some additional trivial conversion (like converting to available
  kernels for `pytorch`, converting to a matrix format for `xgboost`, and slice
  of array for `triton` is handled automatically by the respective wrappers)
- `numpy_to_awkward` (optional): converting the number results back to awkward
  array format. If this is not provided, then a simple `ak.from_numpy`
  conversion takes place.
- `dask_columns` (optional but recommended): Given the inputs to the
  `awkward_to_numpy` method, list the branches required for the inference
  calculation. If not provided, it will attempt to load all branches
  recursively, which may have significant performance penalties.

If the ML tool of choice for your analysis has not been implemented by the
`coffea.mltools` modules, consider constructing your own with the provided
`numpy_call_wrapper` base class in `coffea.mltools`. Aside from the functions
listed above, you will also need to provide the `numpy_call` method to perform
any additional data format conversions, and call the ML tool of choice. If you
think your implementation is general, also consider submitting a PR to the
`coffea` repository!

[triton]: https://catalog.ngc.nvidia.com/orgs/nvidia/containers/tritonserver
[xgboost]: https://xgboost.readthedocs.io/en/stable/
